# Application of Bootstrap samples in Random Forest

In [1]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
import pandas as pd

 <li> Load the boston house dataset </li>

In [2]:
boston = load_boston()
x=boston.data #independent variables
y=boston.target #target variable

In [3]:
print(x.shape)
x[10:]

(506, 13)


array([[2.2489e-01, 1.2500e+01, 7.8700e+00, ..., 1.5200e+01, 3.9252e+02,
        2.0450e+01],
       [1.1747e-01, 1.2500e+01, 7.8700e+00, ..., 1.5200e+01, 3.9690e+02,
        1.3270e+01],
       [9.3780e-02, 1.2500e+01, 7.8700e+00, ..., 1.5200e+01, 3.9050e+02,
        1.5710e+01],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]])

In [4]:
#store all the target values in a dictionary along with row index as key
D={i:y[i] for i in range(0,len(y))} 

### Task: 1
<font color='red'><b>Step 1 Creating samples: </b></font> Randomly create 30 samples from the whole boston data points.
<ol>
<li>Creating each sample: Consider any random 303(60% of 506) data points from whole data set and then replicate any 203 points from the sampled points</li>
<li>Ex: For better understanding of this procedure lets check this examples, assume we have 10 data points [1,2,3,4,5,6,7,8,9,10], first we take 6 data points randomly consider we have selected [4, 5, 7, 8, 9, 3] now we will replciate 4 points from [4, 5, 7, 8, 9, 3], consder they are [5, 8, 3,7] so our final sample will be [4, 5, 7, 8, 9, 3, 5, 8, 3,7]</li>
<li> we create 30 samples like this </li>
<li> Note that as a part of the Bagging when you are taking the random samples make sure each of the sample will have                different set of columns</li>
<li> Ex: assume we have 10 columns for the first sample we will select [3, 4, 5, 9, 1, 2] and for the second sample [7, 9, 1, 4, 5, 6, 2] and so on...</li>
<li> Make sure each sample will have atleast 3 feautres/columns/attributes</li>
</ol>

<font color='red'><b>Step 2 Building High Variance Models on each of the sample and finding train MSE value:</b></font> Build a DecisionTreeRegressor on each of the sample.
<ol><li>Build a regression trees on each of 30 samples.</li>
<li>computed the predicted values of each data point(506 data points) in your corpus.</li>
<li> predicted house price of $i^{th}$ data point $y^{i}_{pred} =  \frac{1}{30}\sum_{k=1}^{30}(\text{predicted value of } x^{i} \text{ with } k^{th} \text{ model})$.</li>
<li>Now calculate the $MSE =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$.</li>
</ol>

<font color='red'><b>Step 3 Calculating the OOB score :</b></font>
<ol>
<li>Computed the predicted values of each data point(506 data points) in your corpus.</li>
<li>Predicted house price of $i^{th}$ data point $y^{i}_{pred} =  \frac{1}{k}\sum_{\text{k= model which was buit on samples not included } x^{i}}(\text{predicted value of } x^{i} \text{ with } k^{th} \text{ model})$.</li>
<li>Now calculate the $OOB Score =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$.</li>
</ol>

### Task: 2
<pre>
<font color='red'><b>Computing CI of OOB Score and Train MSE</b></font>
<ol>
<li> Repeat Task 1 for 35 times, and for each iteration store the Train MSE and OOB score </li>
<li> After this we will have 35 Train MSE values and 35 OOB scores </li>
<li> using these 35 values (assume like a sample) find the confidence intravels of MSE and OOB Score </li>
<li> you need to report CI of MSE and CI of OOB Score </li>
<li> Note: Refer the Central_Limit_theorem.ipynb to check how to find the confidence intravel</li>
</ol>
</pre>
### Task: 3
<pre>
<font color='red'><b>Given a single query point predict the price of house.</b></font>

<li>Consider xq= [0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60] Predict the house price for this point as mentioned in the step 2 of Task 1. </li>
</pre>

In [5]:
columns=boston.feature_names
x=pd.DataFrame(data=boston.data, index=None, columns=columns, dtype=None, copy=False)
y=pd.DataFrame(data=boston.target,columns=['Target'])

In [6]:
from sklearn.tree import DecisionTreeRegressor
import random
def random_select_rows(x_train): 
    c= random.sample(range(0, len(x_train)), int(0.6*len(x_train)))#randomly select 60 percent rows from X
    oob=list(set(np.arange(0,506))-set(c))
    k=np.random.choice(c, 203, replace=False)#sample 203 elements randomly from selected list
    k=[i for i in k]#form a list of all the sampled indecies
    return((c+k),oob)#add both the lists
def random_select_columns(col):#randomly select colum indecies
    return(np.random.choice(col,8,replace=False))

In [7]:
#create 30 Decision trees
dtr1=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr2=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr3=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10) 
dtr4=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr5=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr6=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr7=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr8=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr9=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr10=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr11=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr12=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr13=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10) 
dtr14=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr15=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr16=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr17=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr18=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr19=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr20=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr21=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr22=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr23=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10) 
dtr24=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr25=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr26=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr27=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr28=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr29=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)
dtr30=DecisionTreeRegressor(criterion='mse',random_state=0,min_samples_split=5,max_depth=10)

In [8]:
regressor={1:dtr1,2:dtr2,3:dtr3,4:dtr4,5:dtr5,6:dtr6,7:dtr7,8:dtr8,9:dtr9,10:dtr10 ,11:dtr11,12:dtr12,13:dtr13,14:dtr14,
           15:dtr15,15:dtr15,16:dtr16,17:dtr17,18:dtr18,19:dtr19 ,20:dtr20,21:dtr21,22:dtr22,23:dtr23,
           24:dtr24,25:dtr25,26:dtr26,27:dtr27,28:dtr28 ,29:dtr29,30:dtr30 }

In [9]:
def compute_mse():
    #create 30samples and store therir row and column indecies
    row_list=[]
    col_list=[]
    x_list=[]
    y_list=[]
    oob_list=[]
    x_oob=[]
    y_oob=[]
    for i in range(0,30):
        row,oob=random_select_rows(x)#row indecies
        row_list.append(row)#list of row indecies
        oob_list.append(oob)#list oob indecies
        col=random_select_columns(13)#column indecies
        col_list.append(col)#list of column indecies
        x_list.append(x.iloc [row,col])
        x_oob.append(x.iloc[oob,col])
        y_list.append(y.iloc[row]) 
        y_oob.append(y.iloc[oob])
    
    #fit 30 trees with each sample of dataset
    for i in range(1,31): 
        regressor[i].fit(x_list[i-1], y_list[i-1]) 
        
    Y1=[]   #list of row indecies and predicted values of train data
    Y2=[]   #list of row indecies and predicted values of OOB samples
    for i in range(0,30): 
        y_pred=[] 
        y_oob_pred=[]
        y_pred=regressor[i+1].predict(x_list[i])#predicted values of x_list
        y_oob_pred=regressor[i+1].predict(x_oob[i])
        l=row_list[i]
        for j in range(506):
            Y1.append([l[j],y_pred[j]]) #store predicted value along with its row index
        l2=oob_list[i]
        for k in range(203):
            Y2.append([l2[k],y_oob_pred[k]]) #store predicted value along with its oob index
    
    #Dictionary to store predicted values with row indecies as key and output as values
    D_train={i:[] for i in range(506)}
    D_oob={i:[] for i in range(506)}
    for i in Y1:
        D_train[i[0]].append(i[1])
    for i in Y2:
        D_oob[i[0]].append(i[1])
    
    s1=0
    for i in range(506):       #MSE
        p=np.mean(D_train[i])
        s1+=((D[i]-p)*(D[i]-p))
     
    s2=0
    for i in range(506):      #OOB score
        p=np.mean(D_oob[i])
        s2+=((D[i]-p)*(D[i]-p))
    
    return(s1/506,s2/506,col_list)

# Task: 1

In [10]:
mse,oob_score,col_list=compute_mse()
print('MSE : ',mse)
print('OOB Score : ',oob_score)

MSE :  0.2940923755854412
OOB Score :  12.317726838530405


# Task: 2

In [11]:
mse_ci=[] #MSE CI values
oob_ci=[] #OOb score values
for i in range(35):
    mse,oob_score,col_list=compute_mse()
    mse_ci.append(mse)
    oob_ci.append(oob_score)
print(mse_ci)
print('*'*20)
print(oob_ci)

[0.26563853407383403, 0.2882674546532581, 0.2099446839148508, 0.28255944390734106, 0.2717120810926424, 0.220435242421669, 0.25230474366671335, 0.23302868310099656, 0.24736170048929837, 0.3092819989975252, 0.29601144692576836, 0.26059020959516327, 0.30591348969495125, 0.20206070491386227, 0.2746259886607548, 0.30497432799233454, 0.24507568078432096, 0.2488795173961554, 0.3455256290918091, 0.27716771320022593, 0.23822448179812775, 0.24845726502395618, 0.266492271791186, 0.3057965561507085, 0.23343083478687768, 0.2613958089358012, 0.21130641156578014, 0.2639158580948933, 0.26692833418642564, 0.2345155138041906, 0.2354080012210315, 0.33574548177413094, 0.1886631359867643, 0.25931911995047024, 0.3116431158788051]
********************
[14.430396226682955, 12.251391205851284, 12.970633519345663, 14.184682000746314, 14.701878853616208, 11.497559321918711, 13.127012952017989, 15.451443832711824, 14.101814199658211, 15.375245695082983, 13.554235699218731, 13.890673373988966, 14.307678592913586, 

In [13]:
from sklearn.utils import resample
 
mean_mse=[] #1000 sample means of 30 MSE samples each
mean_oob=[] #1000 sample means of 30 OOB Score samples each
for i in range(1000):
    s1=resample(mse_ci,n_samples=30)
    m1=np.mean(s1)
    mean_mse.append(m1)
    
    s2=resample(oob_ci,n_samples=30)
    m2=np.mean(s2)
    mean_oob.append(m2)

In [16]:
alpha=0.95
p=((1-alpha)/2)*100
l1=np.percentile(mean_mse,p)
p=(alpha+((1-alpha)/2))*100
u1=np.percentile(mean_mse,p)
print('95% CI of mean of MSE : [{} , {}]'.format(l1,u1))

p=((1-alpha)/2)*100
l2=np.percentile(mean_oob,p)
p=(alpha+((1-alpha)/2))*100
u2=np.percentile(mean_oob,p)
print('95% CI of mean of OOB score : [{} , {}]'.format(l2,u2))


95% CI of mean of MSE : [0.2505252551420549 , 0.27584536147067124]
95% CI of mean of OOB score : [13.101408021992242 , 13.825853127206175]


# Task: 3

In [15]:
#input query
xq= [0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60]

y_p=[]
for i in range(30):
    #each tree has input with the same column indecies with which it was trained
    x1=np.array([xq[j] for j in col_list[i]]).reshape(1,-1)
    y_p.append(regressor[i+1].predict(x1))
print('predicted price of the house : ',np.mean(y_p))

predicted price of the house :  20.811565189140325
